In [ ]:
%run network-init.ipynb

# Read some subset to match

In [ ]:
df = pd.read_pickle("dataframe.pickled")

In [ ]:
len(df)

In [ ]:
df = df[df.FonetiskNavn.str.startswith("TT")]

In [ ]:
df = df[df.Køn==False] # oh my god det er grimt, men sådan er pandas syntax

In [ ]:
len(df)

In [ ]:
import recordlinkage

In [ ]:
dfA = df[df.FT==1845]

In [ ]:
len(dfA)

In [ ]:
dfB = df[df.FT==1850]

In [ ]:
len(dfB)

In [ ]:
len(dfA) * len(dfB)

In [ ]:
indexer = recordlinkage.indexing.SortedNeighbourhoodIndex(on="Fødeår", window=5)

In [ ]:
indexer = recordlinkage.indexing.FullIndex()

In [ ]:
index = indexer.index(dfA, dfB)

In [ ]:
len(index)

In [ ]:
def generate_scores(d):
    lots = []
    for (a, b) in d:
        a = dfA.loc[a]
        b = dfB.loc[b]
        lots.append((a.FT, a.Kipnr, a.Løbenr, b.FT, b.Kipnr, b.Løbenr) + score(a,b))
    return lots

In [ ]:
import utils

In [ ]:
with multiprocessing.Pool() as p:
    res = p.map(generate_scores, utils.chunks(index, 1000))

In [ ]:
import itertools

In [ ]:
df_eval = pd.DataFrame(list(itertools.chain(*res)), columns=["a_FT", "a_Kipnr", "a_Løbenr", "b_FT", "b_Kipnr", "b_Løbenr"] + score_columns)

In [ ]:
df_eval

In [ ]:
input_fn = tf.estimator.inputs.pandas_input_fn(x=df_eval, shuffle=False)

In [ ]:
prediction = list(model1.predict(input_fn))

In [ ]:
df.set_index(["FT", "Kipnr", "Løbenr"], inplace=True)

In [ ]:
df.sort_index(inplace=True)

In [ ]:
len(prediction)

In [ ]:
header = "Amt|Herred|Sogn|Navn|Køn|Fødested|Fødeår|Civilstand|Position|Erhverv|Kipnr|Løbenr|Group".split("|")

In [ ]:
import textwrap

In [ ]:
textwrap.shorten("arbasbasb", 5)

In [ ]:
count = 0

for i, d in enumerate(prediction):
    if d["probabilities"][0] < 0.15:
        count += 1
        print(d["probabilities"])
        print(d["classes"])
        r = df_eval.iloc[i]
        s = df.loc[[(r.a_FT,r.a_Kipnr,r.a_Løbenr),
                     (r.b_FT,r.b_Kipnr,r.b_Løbenr)]]
        a = s.iloc[0]
        b = s.iloc[1]
        for i in range(10):
            print("{:10s} {:40s} {:40s}".format(header[i],
                                                textwrap.shorten(str(a[i]),40), 
                                                textwrap.shorten(str(b[i]), 40)))
        print()

In [ ]:
count

In [ ]:
for var in model1.get_variable_names():
    print(var)
    print(model1.get_variable_value(var))